In [1]:
from pathlib import Path
import numpy as np
import os
import pandas as pd
import re
import time

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
print(sys.path)

['/opt/miniconda3/envs/torchtf/lib/python38.zip', '/opt/miniconda3/envs/torchtf/lib/python3.8', '/opt/miniconda3/envs/torchtf/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/torchtf/lib/python3.8/site-packages', '/opt/miniconda3/envs/torchtf/lib/python3.8/site-packages/setuptools/_vendor', '../']


In [28]:
import os
import pandas as pd

input_dir = "../data/can-train-and-test/set_01/train_01"

output_dir = "../data/can-train-and-test/set_01/train_01_merged"
os.makedirs(output_dir, exist_ok=True)

grouped_files = {}

for file in os.listdir(input_dir):
    if file.endswith(".csv"):
        prefix = file.split("-")[0]
        if prefix not in grouped_files:
            grouped_files[prefix] = []
        grouped_files[prefix].append(os.path.join(input_dir, file))

for prefix, files in grouped_files.items():
    df_list = []
    for file_path in files:
        print(f"Reading {file_path}")
        df = pd.read_csv(file_path)

        df['arbitration_id'] = df['arbitration_id'].apply(
            lambda x: int(x, 16) if isinstance(x, str) and all(c in "0123456789ABCDEFabcdef" for c in x) else x
        )

        df['data_field'] = df['data_field'].apply(
            lambda x: bin(int(x, 16))[2:].zfill(64) if isinstance(x, str) and all(c in "0123456789ABCDEFabcdef" for c in x) else x
        )

        required_columns = ['timestamp', 'arbitration_id', 'data_field', 'attack']
        df = df[required_columns].dropna()

        df_list.append(df)
    print(df_list)
    if df_list: 
        merged_df = pd.concat(df_list, ignore_index=True)
        output_file_path = os.path.join(output_dir, f"{prefix}.csv")
        merged_df.to_csv(output_file_path, index=False)
        print(f"Merged files for attack {prefix} into {output_file_path}")


Reading ../data/can-train-and-test/set_01/train_01/attack-free-1.csv
0          193
1          197
2          388
3          455
4          461
          ... 
1952828    842
1952829    249
1952830    409
1952831    413
1952832    501
Name: arbitration_id, Length: 1952833, dtype: int64
0          0011000000000000000000000000010000110000000000...
1          0011000000000000000000000000010000110000000000...
2          0000000000000000000000000000001000000000000000...
3          0000000000000110010111001011100110100010000000...
4          0000000000000000000000000000000000000000000000...
                                 ...                        
1952828    0000000000000000000000000000000000000000000000...
1952829    0000000000000000010000000000000000000000110010...
1952830    0000111111111111000011111111111111110000000000...
1952831    1000000000000000001111111111111000000000000010...
1952832    0000111100001111000000000000000100000000000000...
Name: data_field, Length: 1952833, dtype: o

In [24]:
directory_path = '../data/can-train-and-test/set_01/train_01_merged/accessory.csv'

df = pd.read_csv(directory_path)
# Display the first few rows of the combined DataFrame
filtered_df = df[df['attack'] != 1]
filtered_df

,timestamp,arbitration_id,data_field,attack
0,1.672531e+09,0C1,2000000420000004,0
1,1.672531e+09,0C5,2000000420000004,0
2,1.672531e+09,0F1,34030040,0
3,1.672531e+09,1E5,46FF1AC00000E401,0
4,1.672531e+09,0F9,0000400000000015,0
...,...,...,...,...
433865,1.672531e+09,0C9,0000000A004008,0
433866,1.672531e+09,191,064E064E064E0000,0
433867,1.672531e+09,1A1,00100000000000,0
433868,1.672531e+09,1C3,064E064E00000000,0
